# core

In [ ]:
#| default_exp core

In [ ]:
#| export
import pandas as pd, numpy as np, logging, sys, re, os, torch, json, inspect
from sklearn.tree import DecisionTreeClassifier
from scipy import sparse
from tqdm.auto import tqdm
import torch.nn.functional as F
from itertools import chain
from scipy import sparse
from IPython.display import display
from transformers import AutoTokenizer, PreTrainedTokenizerBase, BatchEncoding
from typing import List, Dict, Union, Optional, Any, Callable
from torch.utils.data import Dataset

from fastcore.dispatch import *
from fastcore.basics import *

In [ ]:
#| hide
from nbdev.showdoc import *
import nbdev; nbdev.nbdev_export()

## `Info`: LOADS METADATA

In [ ]:
#| export
def show_data(x:Dict, n:Optional[int]=10, seed:Optional[int]=None):
    with pd.option_context('display.max_colwidth', None):
        display(pd.DataFrame(x).sample(n, random_state=seed))
        

In [ ]:
#| export
class Info():

    def __init__(self):
        self.tokz, self.info = None, None
        
    @staticmethod
    def _read_txt(fname:str, sep:Optional[str]='->', enc:Optional[str]='latin-1'):
        with open(fname, encoding=enc) as f:
            info = [o[:-1] for o in f]
        return list(zip(*[(o,) if sep is None else o.split(sep, maxsplit=1) for o in info]))

    @staticmethod
    def _read_csv(fname:str):
        df = pd.read_csv(fname).fillna('')
        return [df[c].tolist() for c in df.columns]

    @staticmethod
    def _read_file(fname:str, sep:Optional[str]='->', enc:Optional[str]='latin-1'):
        if fname.endswith(".txt"): 
            return Info._read_txt(fname, sep=sep, enc=enc)
        elif fname.endswith(".csv"): 
            return Info._read_csv(fname)
        else: 
            raise ValueError(f"Invalid filename: {fname}.")
        
    @staticmethod
    def _read_info(fname:str, sep:Optional[str]='->', info_column_names:Optional[List]=None, enc:Optional[str]='latin-1'):
        info = Info._read_file(fname, sep=sep, enc=enc)
        info_column_names = list(range(len(info))) if info_column_names is None else info_column_names
        if len(info_column_names) != len(info): raise ValueError(f'`info_column_names` and `info` should have same number of elements.')
        return {p:q for p,q in zip(info_column_names, info)}

    def read_info(self, fname:Optional[str], sep:Optional[str]='->', info_column_names:Optional[List]=None, enc:Optional[str]='latin-1'):
        self.info = Info._read_info(fname, sep, info_column_names, enc)
        return self.info
    
    def tokenize(self, tokenization_column:Union[int, str], tokenizer:Union[str, PreTrainedTokenizerBase], 
                 max_sequence_length:Optional[int]=None, padding:Optional[bool]=True, return_tensors:Optional[str]=None, 
                 prompt_func:Optional[Callable]=None):
        if self.tokz is None: self.tokz = tokenizer if isinstance(tokenizer, PreTrainedTokenizerBase) else AutoTokenizer.from_pretrained(tokenizer)
        tokenization_column = list(self.info.keys())[0] if tokenization_column is None else tokenization_column
        if tokenization_column is None: logging.info(f'`tokenization_column` not given as input, so value set to {tokenization_column}.')
        if tokenization_column not in self.info: raise ValueError(f'`{tokenization_column}` is invalid `tokenization_column` value.')

        tokenization_text = self.info[tokenization_column] if prompt_func is None else [prompt_func(t) for t in self.info[tokenization_column]]
        self.info.update(self.tokz(tokenization_text, truncation=True, max_length=max_sequence_length, 
                                   padding=padding, return_tensors=return_tensors))
        
        return self.info

    def show_data(self, n:Optional[int]=10, seed:Optional[int]=None):
        with pd.option_context('display.max_colwidth', None):
            display(pd.DataFrame(self.info).sample(n, random_state=seed))

    def __len__(self):
        if self.info is None: return 0
        n_info = [len(v) for v in self.info.values()]
        if len(n_info) == 0: raise ValueError('`info` cannot be empty.')
        if not np.all([o == n_info[0] for o in n_info]): raise ValueError('`info` should contain features with same length.')
        return n_info[0]

    @classmethod
    def from_txt(cls, 
                 fname:str, 
                 sep:Optional[str]='->', 
                 info_column_names:Optional[List]=None, 
                 enc:Optional[str]='latin-1',
                 use_tokenizer:Optional[bool]=False,
                 tokenizer:Optional[Union[str,PreTrainedTokenizerBase]]=None,
                 tokenization_column:Optional[str]=None,
                 max_sequence_length:Optional[int]=None,
                 padding:Optional[bool]=True,
                 return_tensors:Optional[str]=None,
                 prompt_func:Optional[Callable]=None,
                 **kwargs):
        self = cls()
        self.info = self.read_info(fname, sep, info_column_names, enc)
        if use_tokenizer: 
            self.tokenize(tokenization_column, tokenizer, max_sequence_length, padding=padding, return_tensors=return_tensors, 
                          prompt_func=prompt_func)
        return self.info
        

### Example

In [ ]:
fname = f'/home/scai/phd/aiz218323/scratch/datasets/benchmarks/(mapped)LF-WikiSeeAlsoTitles-320K/raw_data/train.raw.txt'
cols = ['identifier', 'input_text']

In [ ]:
fname = f'/home/scai/phd/aiz218323/scratch/datasets/wikipedia/20250123/LF-WikiSeeAlsoTitles-320K/raw_data/train.old.raw.csv'
cols = ['identifier', 'input_text']

In [ ]:
def prompt_func(text):
    prompt = f"""
    Given the title of a wikipedia article and the corresponding categories of that article on wikipedia,
    your task is to predict the titles of all articles which are likely to be listed in the see also section of the
    mentioned article. Output the coma separated list of titles of the articles in the see also section of the
    given article.
    ### Input : 
    ### Title : {text}
    
    #### Task Output :
    #### Predicted title :
    """
    return prompt
    

In [ ]:
from transformers import AutoTokenizer
tokz = AutoTokenizer.from_pretrained('meta-llama/Meta-Llama-3-8B')
tokz.add_special_tokens({"pad_token": "<PAD>"})

1

In [ ]:
info = Info.from_txt(fname, info_column_names=cols, use_tokenizer=True, tokenizer=tokz, tokenization_column=cols[1], 
                     max_sequence_length=128, padding=True, return_tensors='pt', prompt_func=prompt_func)

In [ ]:
tokz.decode(info['input_ids'][10000])

'<|begin_of_text|>\n    Given the title of a wikipedia article and the corresponding categories of that article on wikipedia,\n    your task is to predict the titles of all articles which are likely to be listed in the see also section of the\n    mentioned article. Output the coma separated list of titles of the articles in the see also section of the\n    given article.\n    ### Input : \n    ### Title : Thunderstorm\n    \n    #### Task Output :\n    #### Predicted title :\n    <PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD><PAD>'

In [ ]:
fname = f'/home/scai/phd/aiz218323/scratch/datasets/benchmarks/(mapped)LF-WikiSeeAlsoTitles-320K/raw_data/train.raw.txt'
cols = ['identifier', 'input_text']

info = Info.from_txt(fname, sep='->', info_column_names=cols, use_tokenizer=True, tokenizer='bert-base-uncased', 
                     tokenization_column=cols[0], max_sequence_length=32)
show_data(info, n=5, seed=10)

,identifier,input_text,input_ids,token_type_ids,attention_mask
431204,Certificate_of_Formula_Compliance,Certificate of Formula Compliance,"[101, 8196, 1035, 1997, 1035, 5675, 1035, 12646, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
651497,2017_Sunwolves_season,2017 Sunwolves season,"[101, 2418, 1035, 3103, 12155, 20899, 1035, 2161, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
208048,Hadza_people,Hadza people,"[101, 2018, 4143, 1035, 2111, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
179912,The_Lost_City_(2005_film),The Lost City (2005 film),"[101, 1996, 1035, 2439, 1035, 2103, 1035, 1006, 2384, 1035, 2143, 1007, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
130941,Testability,Testability,"[101, 3231, 8010, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"


In [ ]:
fname = f'/home/scai/phd/aiz218323/scratch/datasets/wikipedia/20250123/LF-WikiSeeAlsoTitles-320K/raw_data/train.old.raw.csv'
cols = ['identifier', 'input_text']

info = Info.from_txt(fname, sep=None, info_column_names=cols, use_tokenizer=True, tokenizer='bert-base-uncased', 
                     tokenization_column=cols[0], max_sequence_length=32)
show_data(info, n=5, seed=10)

,identifier,input_text,input_ids,token_type_ids,attention_mask
431204,Certificate_of_Formula_Compliance,Certificate of Formula Compliance,"[101, 8196, 1035, 1997, 1035, 5675, 1035, 12646, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
651497,2017_Sunwolves_season,2017 Sunwolves season,"[101, 2418, 1035, 3103, 12155, 20899, 1035, 2161, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
208048,Hadza_people,Hadza people,"[101, 2018, 4143, 1035, 2111, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
179912,The_Lost_City_(2005_film),The Lost City (2005 film),"[101, 1996, 1035, 2439, 1035, 2103, 1035, 1006, 2384, 1035, 2143, 1007, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
130941,Testability,Testability,"[101, 3231, 8010, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"


In [ ]:
info = Info()
_ = info.read_info(fname, cols=cols)

In [ ]:
info.show_data(n=10)

,identifier,input_text
247245,20_minutes_(Switzerland),20 minutes (Switzerland)
2305,Isoelectric_point,Isoelectric point
538866,Moldova_in_the_Eurovision_Song_Contest_2013,Moldova in the Eurovision Song Contest 2013
100310,Lemhi_Pass,Lemhi Pass
639082,Order_of_precedence_in_Kelantan,Order of precedence in Kelantan
510129,"RTS,S","RTS,S"
216809,Geering_(automobile),Geering (automobile)
569292,Sole_Survivor_(2013_film),Sole Survivor (2013 film)
175818,Puerto_Rico_National_Cemetery,Puerto Rico National Cemetery
615747,Anna_IllÃ©s,Anna IllÃ©s


In [ ]:
len(info)

693082

## `Filterer`: XC FILTERING

In [ ]:
#| export
class Filterer:

    @staticmethod
    def load_filter(fname:str):
        if fname is not None and os.path.exists(fname): return np.loadtxt(fname, dtype=np.int64)
        
    @staticmethod
    def generate(train_id:List, test_id:List, lbl_id:List, train_lbl:sparse.csr_matrix, test_lbl:sparse.csr_matrix):
        _, train_idx, lbl2train_idx = np.intersect1d(train_id, lbl_id, return_indices=True)
        train_lbl_filterer = np.vstack([train_idx, lbl2train_idx]).T
        
        _, test_idx, lbl2test_idx = np.intersect1d(test_id, lbl_id, return_indices=True)
        test_lbl_filterer = np.vstack([test_idx, lbl2test_idx]).T
        
        train_udx, train_udx2idx = np.unique(train_idx, return_index=True)
        lbl2test_udx, lbl2test_udx2idx = np.unique(lbl2test_idx, return_index=True)
        
        _test_lbl_filterer = train_lbl[train_udx][:, lbl2test_udx].T
        
        rows, cols = _test_lbl_filterer.nonzero()
        test_idx = test_idx[lbl2test_udx2idx[rows]]
        lbl2test_idx = lbl2train_idx[train_udx2idx[cols]]
        
        _test_lbl_filterer = np.vstack([test_idx, lbl2test_idx]).T
        test_lbl_filterer = np.vstack([test_lbl_filterer, _test_lbl_filterer])
    
        return train_lbl_filterer, test_lbl_filterer

    @staticmethod
    def sample(f:np.array, sz:tuple, idx:List):
        f = sparse.coo_matrix((np.full(f.shape[0],1), (f[:, 0], f[:, 1])), shape=sz).tocsr()
        f = f[idx].tocoo()
        return np.vstack([f.row, f.col]).T

    @staticmethod
    def prune(data:sparse.csr_matrix, data_filterer:np.array):
        data = data.copy()
        data[data_filterer[:,0], data_filterer[:,1]] = 0
        data.eliminate_zeros()
        
        idx = np.where(data.getnnz(axis=1) > 0)[0]
        return data[idx], Filterer.sample(data_filterer, data.shape, idx), idx

    @staticmethod
    def apply(data:sparse.csr_matrix, data_filterer:np.array):
        data[data_filterer[:,0], data_filterer[:,1]] = 0
        data.eliminate_zeros()
        return data

        

## Token `IDF`

In [ ]:
#| export
def get_tok_sparse(tokens:List, n_cols:Optional[int]=None):
    n_toks = torch.tensor([len(tok) for tok in tokens])
    tok_ptr = torch.concat([torch.zeros((1,), dtype=torch.long), n_toks.cumsum(dim=0)])
    toks = torch.tensor(list(chain(*tokens)))
    tok_cnt = torch.full((toks.shape[0],), 1, dtype=torch.long)
    m = sparse.csr_matrix((tok_cnt, toks, tok_ptr)) if n_cols is None else sparse.csr_matrix((tok_cnt, toks, tok_ptr), shape=(len(n_toks), n_cols))
    m.sum_duplicates()
    return m

def compute_inv_doc_freq(inputs:sparse.csr_matrix):
    n_docs = inputs.shape[0]
    doc_freq = torch.tensor(inputs.getnnz(axis=0))
    return torch.log((n_docs+1)/(doc_freq+1))+1

def get_tok_idf(dset:Dataset, field:Optional[str]='data_input_ids', n_cols:Optional[int]=None):
    toks = [list(chain(*dset[i][field])) for i in tqdm(range(len(dset)))]
    tok_sparse = get_tok_sparse(toks, n_cols=n_cols)
    return compute_inv_doc_freq(tok_sparse)
    

## Helper

In [ ]:
#| export
def prepare_batch(m, b, m_args=None):
    m_kwargs = inspect.signature(m.forward).parameters
    return BatchEncoding({k:v for k,v in b.items() if k in m_kwargs or (m_args is not None and k in m_args)})
    

In [ ]:
#| export
def store_attr(names=None, self=None, but='', cast=False, store_args=None, is_none=True, **attrs):
    fr = sys._getframe(1)
    args = argnames(fr, True)
    if self: args = ('self', *args)
    else: self = fr.f_locals[args[0]]
    if store_args is None: store_args = not hasattr(self,'__slots__')
    if store_args and not hasattr(self, '__stored_args__'): self.__stored_args__ = {}
    anno = annotations(self) if cast else {}
    if names and isinstance(names,str): names = re.split(', *', names)
    ns = names if names is not None else getattr(self, '__slots__', args[1:])
    added = {n:fr.f_locals[n] for n in ns}
    attrs = {**attrs, **added}
    if isinstance(but,str): but = re.split(', *', but)
    attrs = {k:v for k,v in attrs.items() if k not in but}
    return _store_attr(self, anno, is_none, **attrs)
    

In [ ]:
#| export
def _store_attr(self, anno, is_none, **attrs):
    stored = getattr(self, '__stored_args__', None)
    for n,v in attrs.items():
        if n in anno: v = anno[n](v)
        if is_none or v is not None: setattr(self, n, v)
        if stored is not None: stored[n] = v
       

In [ ]:
#| export
def get_attr(x, attr:str):
    for a in attr.split('.'): x = getattr(x, a)
    return x

In [ ]:
#| export
def sorted_metric(keys:List, order:Optional[Dict]=None):
    order = {o.split('@')[0]:i for i,o in enumerate(keys)}
    def _suffix(x): return (int(x.split('_')[0]) , x.split('_')[1]) if '_' in x else (int(x),)
    def _key_fn(x): return order[x[0]], _suffix(x[1])
    def key_fn(x): return _key_fn(x.split('@'))
    return sorted(keys, key=key_fn)

def display_metric(metrics, remove_prefix:Optional[bool]=True, order:Optional[List]=None, scale:Optional[int]=100.0):
    metrics = {k.split('_', maxsplit=1)[1]:v for k,v in metrics.items()} if remove_prefix else metrics
    metric_keys, other_keys = sorted_metric([k for k in metrics if '@' in k], order), [k for k in metrics if '@' not in k]
    
    from IPython.display import display
    with pd.option_context('display.precision',4,'display.max_colwidth',None,'display.max_columns',None):
        metric,other = pd.DataFrame([metrics])[metric_keys]*scale, pd.DataFrame([metrics])[other_keys]
        display(pd.concat([metric, other], axis=1))
        

In [ ]:
#| export
def get_tensor_statistics(x:torch.Tensor):
    c = ['mean', 'std', '25', '50', '75']
    s = torch.cat([x.float().mean(dim=0, keepdim=True), 
                   x.float().std(dim=0, keepdim=True),
                   torch.quantile(x.float(), torch.tensor([0.25, 0.5, 0.75]))])
    return pd.DataFrame([s.tolist()], columns=c)

def total_recall(inp_idx:torch.Tensor, n_inp:torch.Tensor, targ:sparse.csr_matrix, filterer:sparse.csr_matrix):
    val, ptr = torch.ones(len(inp_idx)), torch.cat([torch.zeros(1, dtype=torch.int64), n_inp.cumsum(0)])
    inp = sparse.csr_matrix((val,inp_idx,ptr), shape=targ.shape); inp.sum_duplicates(); inp.data[:] = 1
    if filterer is not None: inp, targ = Filterer.apply(inp, filterer), Filterer.apply(targ, filterer)
    sc = inp.multiply(targ)/(targ.getnnz(axis=1)[:, None]*targ.shape[0])
    return sc.sum(), sc
    

In [ ]:
#| export
def get_best_model(mdir:str, pat:Optional[str]=r'^checkpoint-(\d+)'):
    nm = sorted([int(re.match(pat, o).group(1)) for o in os.listdir(mdir) if re.match(pat, o)])[-1]
    fname = f'{mdir}/checkpoint-{nm}/trainer_state.json'
    with open(fname, 'r') as file: mname = json.load(file)['best_model_checkpoint']
    return f'{mdir}/checkpoint-{nm}' if mname is None else mname
    

In [ ]:
#| export
def get_output_sparse(pred_idx, pred_ptr, pred_score, targ_idx, targ_ptr, n_lbl):
    n_data = pred_ptr.shape[0]
    
    pred_ptr = torch.cat([torch.zeros((1,), dtype=torch.long), pred_ptr.cumsum(dim=0)])
    
    targ_ptr = torch.cat([torch.zeros((1,), dtype=torch.long), targ_ptr.cumsum(dim=0)])
    targ_score = torch.ones((targ_idx.shape[0],), dtype=torch.long)
    
    pred = sparse.csr_matrix((pred_score,pred_idx,pred_ptr), shape=(n_data, n_lbl))
    targ = sparse.csr_matrix((targ_score,targ_idx,targ_ptr), shape=(n_data, n_lbl))
    return pred, targ


In [ ]:
#| export
def get_output(data_lbl, pred_lbl):
    output = {
        'targ_idx': torch.tensor(data_lbl.indices),
        'targ_ptr': torch.tensor([q-p for p,q in zip(data_lbl.indptr, data_lbl.indptr[1:])]),
        'pred_idx': torch.tensor(pred_lbl.indices),
        'pred_ptr': torch.tensor([q-p for p,q in zip(pred_lbl.indptr, pred_lbl.indptr[1:])]),
        'pred_score': torch.tensor(pred_lbl.data),
    }
    return output

In [ ]:
#| export
def load_config(fname, key):
    with open(fname, 'r') as file:
        return json.load(file)[key]
        

## `ScoreFusion`

In [ ]:
#| export
class ScoreFusion():
    
    def __init__(self, prop:Optional[np.array]=None, max_depth:Optional[int]=7):
        self.clf, self.prop = DecisionTreeClassifier(max_depth=max_depth), prop
        
    def sample(self, 
               score_a:sparse.csr_matrix, 
               score_b:sparse.csr_matrix, 
               targ:Optional[sparse.csr_matrix]=None, 
               n_samples:Optional[int]=None):
        if n_samples is not None and n_samples > 0 and n_samples < score_a.shape[0]:
            rnd_idx = np.random.permutation(score_a.shape[0])[:n_samples]
            score_a, score_b = score_a[rnd_idx], score_b[rnd_idx]
            targ = targ if targ is None else targ[rnd_idx]
        return score_a, score_b, targ
    
    def get_inp(self, row_idx, col_idx, score_a, score_b):
        inp = [np.array(o[row_idx,col_idx]).reshape(-1, 1) for o in [score_a, score_b]]
        if self.prop is not None: inp.append(self.prop[col_idx].reshape(-1,1))
        return np.hstack(inp)
    
    def prepare_inputs(self, score_a:sparse.csr_matrix, score_b:sparse.csr_matrix, 
                       targ:Optional[sparse.csr_matrix]=None, n_samples:Optional[int]=None):
        if n_samples is not None:
            score_a, score_b, targ = self.sample(score_a, score_b, targ, n_samples)
        row_idx, col_idx = score_b.nonzero()
        inp = self.get_inp(row_idx, col_idx, score_a, score_b)
        targ = targ if targ is None else np.array(targ[row_idx, col_idx]).ravel()
        return inp, targ, (row_idx,col_idx)
        
    def fit(self, score_a:sparse.csr_matrix, score_b:sparse.csr_matrix, targ:Optional[sparse.csr_matrix]=None, 
            n_samples:Optional[int]=None):
        score, targ, _ = self.prepare_inputs(score_a, score_b, targ, n_samples)
        self.clf.fit(score, targ)

    def predict(self, score_a:sparse.csr_matrix, score_b:sparse.csr_matrix, beta:Optional[int]=1):
        inp, _, (row_idx, col_idx) = self.prepare_inputs(score_a, score_b)
        res = score_b.copy()
        res[row_idx,col_idx] = self.clf.predict_proba(inp)[:, 1]
        return beta*(res+score_a)+score_b
    

## `Sparse`

In [ ]:
#| export
def retain_randk(matrix:sparse.csr_matrix, topk:Optional[int]=3):
    data, indices, indptr = [], [], np.zeros_like(matrix.indptr)
    for i,row in tqdm(enumerate(matrix), total=matrix.shape[0]):
        if row.nnz > 0:
            idx = np.random.randint(row.nnz, size=topk)
            ind, d = row.indices[idx], row.data[idx]
            indptr[i+1] = indptr[i] + topk
            indices.append(ind); data.append(d)
        else:
            indptr[i+1] = indptr[i]
    
    data = np.hstack(data)
    indices = np.hstack(indices)
    
    o = sparse.csr_matrix((data, indices, indptr), dtype=matrix.dtype)
    o.sort_indices()
    return o
    

## `Robustness Analysis`

In [ ]:
#| export
def random_topk(data_lbl, topk=5):
    data,indices,indptr = [],[],[0]
    for i,j in tqdm(zip(data_lbl.indptr, data_lbl.indptr[1:]), total=data_lbl.shape[0]):
        idx = np.random.permutation(j-i)[:topk]
        data.append(data_lbl.data[i:j][idx])
        indices.append(data_lbl.indices[i:j][idx])
        indptr.append(indptr[-1]+len(idx))
    data = np.hstack(data)
    indices = np.hstack(indices)
    indptr = np.array(indptr)

    o = sparse.csr_matrix((data,indices,indptr), shape=data_lbl.shape, dtype=np.float32)
    o.sort_indices()
    o.eliminate_zeros()
    return o
    

In [ ]:
#| export
def robustness_analysis(block, meta_name:str, analysis_type:str='missing', pct:float=0.5, topk:int=3):
    data_meta = random_topk(block.test.dset.meta[f'{meta_name}_meta'].data_meta, topk=topk)
    
    if analysis_type == 'noise':
        mask = np.random.rand(len(data_meta.data)) < pct
        data_meta.indices[mask] = np.random.randint(block.n_lbl, size=mask.sum())
        data_meta.sort_indices()
    elif analysis_type == 'missing':
        data_meta.data[np.random.rand(len(data_meta.data)) < pct] = 0
        data_meta.eliminate_zeros()
    else:
        raise ValueError(f'Invalid `analysis_type`: {analysis_type}.')
        
    lbl_meta = block.test.dset.meta[f'{meta_name}_meta'].lbl_meta
    block.test.dset.meta[f'{meta_name}_meta'].update_meta_matrix(data_meta, lbl_meta)
    return block
    